In [1]:
# Forked from http://bbengfort.github.io/tutorials/2016/05/19/text-classification-nltk-sckit-learn.html
import string

from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk import pos_tag
from re import sub
from sklearn.base import BaseEstimator, TransformerMixin
from collections import defaultdict

class NLTKPreprocessor(BaseEstimator, TransformerMixin):

    def __init__(self, stopwords=None, punct=None, lower=True, strip=True):
        self.lower      = lower
        self.strip      = strip
        self.stopwords  = stopwords or set(sw.words('english'))
        self.punct      = punct or set(string.punctuation)
        self.lemmatizer = WordNetLemmatizer()

    def fit(self, X, y=None):
        return self

    def inverse_transform(self, X):
        return [" ".join(doc) for doc in X]

    def transform(self, X):
        return [
            list(self.tokenize(doc)) for doc in X
        ]

    def tokenize(self, document):
        # Break the document into sentences
        for sent in sent_tokenize(document):
            # Break the sentence into part of speech tagged tokens
            for token, tag in pos_tag(wordpunct_tokenize(sent)):
                # Apply preprocessing to the token
                token = token.lower() if self.lower else token
                token = sub("[^a-z]", "", token) if self.strip else token
                
                # If stopword, ignore token and continue
                if token in self.stopwords:
                    continue

                # If punctuation, ignore token and continue
                if all(char in self.punct for char in token):
                    continue

                # Lemmatize the token and yield
                lemma = self.lemmatize(token, tag)
                
                if lemma not in self.stopwords:
                    yield lemma

    def lemmatize(self, token, tag):
        tag = {
            'N': wn.NOUN,
            'V': wn.VERB,
            'R': wn.ADV,
            'J': wn.ADJ
        }.get(tag[0], wn.NOUN)

        return self.lemmatizer.lemmatize(token, tag)

In [2]:
swords = ["a", "able", "about", "above", "abst", "accordance", "according", "accordingly", "across", "act", "actually", "added", "adj", "adopted", "affected", "affecting", "affects", "after", "afterwards", "again", "against", "ah", "all", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "apparently", "approximately", "are", "aren", "arent", "arise", "around", "as", "aside", "ask", "asking", "at", "auth", "available", "away", "awfully", "b", "back", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "between", "beyond", "biol", "both", "brief", "briefly", "but", "by", "c", "ca", "came", "can", "cannot", "cant", "cause", "causes", "certain", "certainly", "click here", "click me", "co", "com", "come", "comes", "contain", "containing", "contains", "could", "couldnt", "d", "date", "did", "didn", "didnt", "different", "do", "does", "doesn", "doesnt", "doing", "don", "done", "dont", "down", "downwards", "due", "during", "e", "each", "ed", "edu", "effect", "eg", "eight", "eighty", "either", "else", "elsewhere", "email", "e-mail", "e-mail address", "email address", "end", "ending", "enough", "especially", "et", "et-al", "etc", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "except", "f", "far", "few", "ff", "fifth", "first", "five", "fix", "followed", "following", "follows", "for", "former", "formerly", "forth", "found", "four", "from", "further", "furthermore", "g", "gave", "get", "gets", "getting", "give", "given", "gives", "giving", "go", "goes", "gone", "got", "gotten", "h", "had", "happens", "hardly", "has", "hasn", "hasnt", "have", "haven", "havent", "having", "he", "hed", "hell", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "hereupon", "hers", "herself", "hes", "hi", "hid", "him", "himself", "his", "hither", "home", "how", "howbeit", "however", "hundred", "i", "id", "ie", "if", "ill", "im", "immediate", "immediately", "importance", "important", "in", "inc", "indeed", "index", "information", "instead", "into", "invention", "inward", "is", "isn", "isnt", "it", "itd", "itll", "its", "itself", "ive", "j", "just", "k", "keep", "keeps", "kept", "keys", "kg", "km", "know", "known", "knows", "l", "largely", "last", "lately", "later", "latter", "latterly", "least", "less", "lest", "let", "lets", "like", "liked", "likely", "line", "little", "ll", "look", "looking", "looks", "ltd", "m", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "million", "ml", "more", "moreover", "more published", "most", "mostly", "much", "mug", "must", "my", "myself", "n", "na", "name", "namely", "nay", "nd", "near", "nearly", "necessarily", "necessary", "need", "needs", "neither", "never", "nevertheless", "new", "next", "nine", "ninety", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "now", "nowhere", "nt", "o", "obtain", "obtained", "obviously", "odd", "of", "off", "often", "oh", "ok", "okay", "old", "omitted", "on", "once", "one", "ones", "only", "onto", "or", "ord", "other", "others", "otherwise", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "owing", "own", "p", "page", "pages", "part", "particular", "particularly", "past", "per", "perhaps", "placed", "please", "plus", "pm", "poorly", "possible", "possibly", "potentially", "pp", "predominantly", "present", "previously", "primarily", "print me", "print this", "probably", "promptly", "proud", "provides", "published", "put", "q", "que", "quickly", "quite", "qv", "r", "ran", "rather", "rd", "re", "readily", "read more", "really", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "respectively", "resulted", "resulting", "results", "right", "run", "s", "said", "same", "saw", "say", "saying", "says", "sec", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sent", "seven", "several", "shall", "she", "shed", "shell", "shes", "should", "shouldn", "shouldnt", "show", "showed", "shown", "showns", "shows", "significant", "significantly", "similar", "similarly", "since", "six", "slightly", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "specifically", "specified", "specify", "specifying", "state", "states", "still", "stop", "street", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "t", "take", "taken", "taking", "tell", "tends", "th", "than", "thank", "thanks", "thanx", "that", "thatll", "thats", "thatve", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "therell", "thereof", "therere", "theres", "thereto", "thereupon", "thereve", "these", "they", "theyd", "theyll", "theyre", "theyve", "think", "this", "those", "thou", "though", "thoughh", "thousand", "throug", "through", "throughout", "thru", "thus", "til", "tip", "to", "together", "too", "took", "toward", "towards", "tried", "tries", "truly", "try", "trying", "ts", "twice", "two", "u", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "up", "updated", "upon", "ups", "us", "use", "used", "useful", "usefully", "usefulness", "user", "uses", "using", "usually", "v", "value", "various", "ve", "very", "via", "viz", "vol", "vols", "vs", "w", "want", "wants", "was", "wasn", "wasnt", "way", "we", "wed", "welcome", "well", "went", "were", "werent", "weve", "what", "whatever", "whatll", "whats", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "wholl", "whom", "whomever", "whos", "whose", "why", "widely", "will", "willing", "wish", "with", "within", "without", "won", "wont", "words", "world", "would", "wouldn", "wouldnt", "www", "x", "y", "yes", "yet", "you", "youd", "youll", "your", "youre", "yours", "yourself", "yourselves", "youve", "z"]

In [3]:
pro = NLTKPreprocessor(stopwords=swords)

In [4]:
text = "Justin Bieber has showed off some silky soccer skills during a training session with FC Barcelona. The 'Never Say Never' hitmaker turned on his football talent at the Catalan club's training ground on Monday (21.11.16) as he had a kick about with the Spanish giants' Brazilian players Neymar and Rafinha. He tweeted afterwards: \Great day\ Justin's skills certainly seemed to impress Brazil international Neymar, who took to Twitter to share a picture of the 22-year-old star trying to take the ball past him. He wrote: \Impossible to dribble this defender...@justinbieber\ Rafinha also appeared to enjoy the club's special guest, as he took to Twitter to share a picture with the 'Company' hitmaker. He wrote: \Great visit today! Nice to see you @justinbieber (sic)\ The pop star also retweeted a message from the official Barcelona account which posted a video of his \mad skills\ in action, including him burying a shot into the bottom corner of the goal. It read: \Veratile @justinbieber shows off mad #SkillsFCB with @Rafinha & @NeymarJr both on attack and in goal! (sic)\ Justin took part in the training session ahead of his show in the Spanish city's Palau Sant Jordi sporting arena on Tuesday night (22.11.16), part of his 138-date 'Purpose World Tour'. He will head to Madrid on Wednesday (23.11.16), where he will perform at the Barclaycard Center Madrid. It's not the first time Justin has rubbed shoulders with Barca greats. In 2011, he also paid a visit to the club and enjoyed a chat with Lionel Messi and Andres Iniesta. Justin Bieber's skills at Barcelona training session"
x = defaultdict(int)
for t in pro.tokenize(text):
    x[t] += 1
print(x)

defaultdict(<class 'int'>, {'shoulder': 1, 'retweeted': 1, 'day': 1, 'sic': 2, 'skillsfcb': 1, 'visit': 2, 'spanish': 2, 'hitmaker': 2, 'tour': 1, 'tweet': 1, 'fc': 1, 'bottom': 1, 'include': 1, 'football': 1, 'player': 1, 'kick': 1, 'palau': 1, 'city': 1, 'pay': 1, 'barca': 1, 'ahead': 1, 'enjoy': 2, 'night': 1, 'neymar': 2, 'impress': 1, 'barclaycard': 1, 'twitter': 2, 'lionel': 1, 'account': 1, 'session': 3, 'monday': 1, 'attack': 1, 'action': 1, 'great': 3, 'brazilian': 1, 'post': 1, 'iniesta': 1, 'andres': 1, 'talent': 1, 'rub': 1, 'share': 2, 'purpose': 1, 'shot': 1, 'company': 1, 'appear': 1, 'chat': 1, 'picture': 2, 'special': 1, 'arena': 1, 'year': 1, 'mad': 2, 'madrid': 2, 'center': 1, 'training': 4, 'message': 1, 'international': 1, 'video': 1, 'sport': 1, 'wednesday': 1, 'official': 1, 'sant': 1, 'pop': 1, 'veratile': 1, 'justinbieber': 3, 'ground': 1, 'brazil': 1, 'bury': 1, 'messi': 1, 'jordi': 1, 'tuesday': 1, 'ball': 1, 'dribble': 1, 'corner': 1, 'today': 1, 'goal': 2, 

In [5]:
import operator
for key, value in sorted(x.items(), reverse=True, key=operator.itemgetter(1)):
    print(key, value)

justin 5
training 4
skill 4
session 3
great 3
justinbieber 3
club 3
rafinha 3
barcelona 3
sic 2
visit 2
spanish 2
hitmaker 2
enjoy 2
neymar 2
twitter 2
share 2
picture 2
mad 2
madrid 2
goal 2
bieber 2
star 2
write 2
shoulder 1
retweeted 1
day 1
skillsfcb 1
tour 1
tweet 1
fc 1
bottom 1
include 1
football 1
player 1
kick 1
palau 1
city 1
pay 1
barca 1
ahead 1
night 1
impress 1
barclaycard 1
lionel 1
account 1
monday 1
attack 1
action 1
brazilian 1
post 1
iniesta 1
andres 1
talent 1
rub 1
purpose 1
shot 1
company 1
appear 1
chat 1
special 1
arena 1
year 1
center 1
message 1
international 1
video 1
sport 1
wednesday 1
official 1
sant 1
pop 1
veratile 1
ground 1
brazil 1
bury 1
messi 1
jordi 1
tuesday 1
ball 1
dribble 1
corner 1
today 1
giant 1
soccer 1
impossible 1
turn 1
silky 1
guest 1
time 1
defender 1
read 1
nice 1
head 1
catalan 1
neymarjr 1
perform 1
